In [1]:
from timeit import default_timer as timer
import pandas as pd
import scipy.stats
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
%matplotlib inline
import time
import timeit
from math import e

In [2]:
%cd "D:\USC\4th semester\DSO562 fraud"

D:\USC\4th semester\DSO562 fraud


In [3]:
data=pd.read_csv('card transactions.csv', converters={'Merchnum': lambda x: str(x)})

In [4]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [99]:
data.shape

(96753, 10)

In [5]:
# select only Transtype is P
data=data[data['Transtype']=='P']

In [13]:
data.shape

(96398, 10)

In [5]:
# remove one outlier
data['Amount'].max()

3102045.53

In [6]:
data.drop(data[data['Amount']==data['Amount'].max()].index, inplace = True)

In [68]:
data.shape

(96397, 10)

In [7]:
data['Fraud'].value_counts()

0    95338
1     1059
Name: Fraud, dtype: int64

In [51]:
# 3198 missing value
data['Merchnum'].value_counts()

930090121224     9310
                 3198
5509006296254    2131
9900020006406    1714
602608969534     1092
                 ... 
6822404702533       1
19904672330         1
300600250           1
4900000000445       1
5954082580007       1
Name: Merchnum, Length: 13091, dtype: int64

In [7]:
data['Merchnum']=data['Merchnum'].str.strip().replace('',np.nan)

In [8]:
data[data['Merchnum'].isna()]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
115,116,5142182128,2006-01-03,NaN,GRAINGER #973,IL,60089.0,P,327.34,0
135,136,5142126842,2006-01-03,NaN,AUTOMATED OFFICE PRODU,MD,20706.0,P,2110.00,0
168,169,5142132574,2006-01-03,NaN,ROLL CALL NEWSPAPER,DC,20001.0,P,104.69,0
201,202,5142140436,2006-01-03,NaN,MONTGOMERY COLLEGE-PHONE,MD,20850.0,P,266.00,0
257,258,5142257292,2006-01-04,NaN,ARGENT CHEMICAL LABS,WA,98052.0,P,240.00,0
...,...,...,...,...,...,...,...,...,...,...
96621,96622,5142253938,2006-12-30,NaN,OREGON ENVIRONMENTAL COUN,OR,97204.0,P,40.00,0
96651,96652,5142151962,2006-12-30,NaN,RETAIL DEBIT ADJUSTMENT,NaN,NaN,P,125.00,0
96689,96690,5142166019,2006-12-30,NaN,ADVANCED SEC & CONTROL,GA,30340.0,P,70.50,0
96720,96721,5142178272,2006-12-31,NaN,TRULOGIC SYSTEMS,ON,NaN,P,343.42,0


In [8]:
# there are no zero Merchnum values
data[data['Merchnum']==0]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud


In [9]:
#Fill in missing values (Zip, Merchnum, State)
merchdes_merchnum = {}
for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
    if merchdes not in merchdes_merchnum :
        merchdes_merchnum[merchdes] = data.loc[index, 'Merchnum']

In [10]:
# fill in by mapping with Merch description
data['Merchnum'] = data['Merchnum'].fillna(data['Merch description'].map(merchdes_merchnum))


In [11]:
# assign unknown for adjustments transactions
data['Merchnum'] = data['Merchnum'].mask(data['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merchnum'] = data['Merchnum'].mask(data['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')
data['Merchnum'] = data['Merchnum'].mask(data['Merchnum'].isna(), 'unknown')

In [12]:
data.sample(10)

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
14364,14365,5142148452,2006-02-28,930090121224,FEDEX SHP 02/22/10 AB#,TN,38118.0,P,6.74,0
9306,9307,5142111930,2006-02-09,602608969534,FISHER SCI ATL,GA,30091.0,P,143.80,0
71228,71229,5142190387,2006-09-07,6899688962417,ALLIED TRAILER SUPPLY,CA,95815.0,P,79.95,0
69285,69286,5142299705,2006-08-31,930090121224,FEDEX SHP 08/28/10 AB#,TN,38118.0,P,3.74,0
24052,24053,5142149874,2006-04-01,1754000282045,PAGENET*PAGING NETWORK,TX,75252.0,P,165.10,0
62540,62541,5142239684,2006-08-11,unknown,RETAIL CREDIT ADJUSTMENT,NaN,NaN,P,453.00,0
19285,19286,5142176897,2006-03-16,465968098337,HPSHOPPING COM,CA,95051.0,P,499.00,0
19252,19253,5142223373,2006-03-16,8000326863954,WRIGHT BROTHERS INC.,OH,45237.0,P,214.00,0
40381,40382,5142148297,2006-05-31,2094330000009,FRANKLIN COVEY COMPANY,UT,84119.0,P,33.96,0
87781,87782,5142195612,2006-11-17,9070100084998,SKC INC,PA,15330.0,P,800.00,0


In [12]:
# dict for mapping
zip_state = {}
for index, zip5 in data[data['Merch zip'].notnull()]['Merch zip'].items():
    if zip5 not in zip_state:
        zip_state[zip5] = data.loc[index, 'Merch state']


In [13]:
zip_state['00926'] = 'PR'
zip_state['00929'] = 'PR'
zip_state['00934'] = 'PR'
zip_state['00902'] = 'PR'
zip_state['00738'] = 'PR'
zip_state['90805'] = 'CA'
zip_state['76302'] = 'TX'
zip_state['00914'] = 'PR'
zip_state['95461'] = 'CA'
zip_state['00680'] = 'PR'
zip_state['00623'] = 'PR'
zip_state['00726'] = 'PR'
zip_state['00936'] = 'PR'
zip_state['12108'] = 'NY'
zip_state['00791'] = 'PR'
zip_state['00907'] = 'PR'
zip_state['00922'] = 'PR'
zip_state['00920'] = 'PR'
zip_state['00801'] = 'VI'
zip_state['31040'] = 'GA'
zip_state['41160'] = 'KY'
zip_state['00681'] = 'PR'

In [14]:
merchnum_state = {}
for index, merchnum in data[data['Merchnum'].notnull()]['Merchnum'].items():
    if merchnum not in merchnum_state :
        merchnum_state [merchnum] = data.loc[index, 'Merch state']

merchdes_state = {}
for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
    if merchdes not in merchdes_state :
        merchdes_state [merchdes] = data.loc[index, 'Merch state']


In [15]:
# fill in by mapping with zip, merchnum and merch description
data['Merch state'] = data['Merch state'].fillna(data['Merch zip'].map(zip_state))
data['Merch state'] = data['Merch state'].fillna(data['Merchnum'].map(merchnum_state))
data['Merch state'] = data['Merch state'].fillna(data['Merch description'].map(merchdes_state))

# assign unknown for adjustments transactions
data['Merch state'] = data['Merch state'].mask(data['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merch state'] = data['Merch state'].mask(data['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')

In [16]:
data['Merch state'].count()

96308

In [16]:
merchnum_zip = {}
for index, merchnum in data[data['Merchnum'].notnull()]['Merchnum'].items():
    if merchnum not in merchnum_zip :
        merchnum_zip [merchnum] = data.loc[index, 'Merch zip']

merchdes_zip = {}
for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
    if merchdes not in merchdes_zip :
        merchdes_zip [merchdes] = data.loc[index, 'Merch zip']

# fill in by mapping with merchnum and merch description
data['Merch zip'] = data['Merch zip'].fillna(data['Merchnum'].map(merchnum_zip))
data['Merch zip'] = data['Merch zip'].fillna(data['Merch description'].map(merchdes_zip))

# assign unknown for adjustments transactions
data['Merch zip'] = data['Merch zip'].mask(data['Merch zip'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merch zip'] = data['Merch zip'].mask(data['Merch zip'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')

# fill the rest with unknown
data = data.fillna('unknown')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96752
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Recnum             96397 non-null  int64  
 1   Cardnum            96397 non-null  int64  
 2   Date               96397 non-null  object 
 3   Merchnum           96397 non-null  object 
 4   Merch description  96397 non-null  object 
 5   Merch state        96397 non-null  object 
 6   Merch zip          96397 non-null  object 
 7   Transtype          96397 non-null  object 
 8   Amount             96397 non-null  float64
 9   Fraud              96397 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 10.6+ MB


In [20]:
data.sample(10)

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
71905,71906,5142131548,2006-09-11,930090121224,FEDEX SHP 09/07/10 AB#,TN,38118,P,3.67,0
68265,68266,5142190147,2006-08-29,930090121224,FEDEX SHP 08/23/10 AB#,TN,38118,P,3.62,0
631,632,5142272511,2006-01-05,679679721335,ORACLE DIRECT MKTG,CA,94065,P,1180.86,0
61995,61996,5142297710,2006-08-10,930090121224,FEDEX SHP 04/14/10 AB#,TN,38118,P,3.62,0
78375,78376,5142123782,2006-09-28,930090121224,FEDEX SHP 09/20/10 AB#,TN,38118,P,3.62,0
28779,28780,5142259479,2006-04-20,4503082476300,OFFICE DEPOT #1082,MD,20763,P,173.24,0
76594,76595,5142170025,2006-09-22,9900000029606,GSA/CUST SUPPLY CTR 82,IL,60482,P,1348.59,0
55874,55875,5142236799,2006-07-24,62007675337,RITZ CAMERA CENTER #1175,PA,19103,P,9.59,0
1645,1646,5142147797,2006-01-10,2603600060593,CHAMPION TROPHIES INC,DC,20001,P,584.00,0
26865,26866,5142267882,2006-04-12,465094661332,AGILENT,GA,30348,P,1554.00,0


In [115]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96752
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Recnum             96397 non-null  int64  
 1   Cardnum            96397 non-null  int64  
 2   Date               96397 non-null  object 
 3   Merchnum           96397 non-null  object 
 4   Merch description  96397 non-null  object 
 5   Merch state        96397 non-null  object 
 6   Merch zip          96397 non-null  object 
 7   Transtype          96397 non-null  object 
 8   Amount             96397 non-null  float64
 9   Fraud              96397 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 10.6+ MB


In [17]:
data['Date'] =  pd.to_datetime(data.Date, format = '%Y-%m-%d')

In [18]:
data['dow']=data['Date'].apply(lambda x: calendar.day_name[x.weekday()])

In [19]:
data['card_merch']=data['Cardnum'].astype(str)+data['Merchnum'].astype(str)
data['card_des']=data['Cardnum'].astype(str)+data['Merch description']
data['card_state']=data['Cardnum'].astype(str)+data['Merch state']
data['card_zip']=data['Cardnum'].astype(str)+data['Merch zip'].astype(str)
data['merch_des']=data['Merchnum'].astype(str)+data['Merch description']
data['merch_state']=data['Merchnum'].astype(str)+data['Merch state']
data['merch_zip']=data['Merchnum'].astype(str)+data['Merch zip'].astype(str)
data['des_state']=data['Merch description']+data['Merch state']
data['des_zip']=data['Merch description']+data['Merch zip'].astype(str)

In [21]:
data.columns

Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud', 'dow',
       'card_merch', 'card_des', 'card_state', 'card_zip', 'merch_des',
       'merch_state', 'merch_zip', 'des_state', 'des_zip'],
      dtype='object')

In [20]:
df1=data.copy()
final=df1.copy()
df1['check_date']=df1['Date']
df1['check_record']=df1['Recnum']
final.shape

(96397, 20)

In [21]:
entities=['Merch description',
       'card_merch', 'card_des', 'card_state', 'card_zip', 'merch_des',
       'merch_state', 'des_state', 'des_zip']

In [22]:
start = timeit.default_timer()
for entity in entities:    
    try: print('Run time for the last entity ----------------- {}s'.format(timeit.default_timer() - st))
    except: print('')
    st = timeit.default_timer()    
    ## Day-since variables:   
    df_l = df1[['Recnum', 'Date', entity]]
    df_r = df1[['check_record', 'check_date', entity, 'Amount']]    
    temp = pd.merge(df_l, df_r, left_on = entity, right_on = entity)   
    temp1 = temp[temp.Recnum > temp.check_record][['Recnum','Date','check_date']]\
                                                   .groupby('Recnum')[['Date', 'check_date']].last()
    mapper = (temp1.Date - temp1.check_date).dt.days
    final[entity + '_day_since'] = final.Recnum.map(mapper)
    final[entity + '_day_since'].fillna((final.Date - pd.to_datetime('2010-01-01')).dt.days, inplace = True)
    print('\n' + entity + '_day_since ---> Done')   
    temp1=0
    ## Frequency & Amount variables:   
    for time in [0,1,3,7,14,30]:       
        temp2 = temp[(temp.check_date >= (temp.Date - dt.timedelta(time))) &\
                       (temp.Recnum >= temp.check_record)][['Recnum', entity, 'Amount']]       
        col_name = entity + '_count_' + str(time)    
        mapper2 = temp2.groupby('Recnum')[entity].count()      
        final[col_name] = final.Recnum.map(mapper2)        
        print(col_name + ' ---> Done')       
        final[entity + '_avg_' + str(time)] = final.Recnum.map(temp2.groupby('Recnum')['Amount'].mean())
        final[entity + '_max_' + str(time)] = final.Recnum.map(temp2.groupby('Recnum')['Amount'].max())
        final[entity + '_med_' + str(time)] = final.Recnum.map(temp2.groupby('Recnum')['Amount'].median())
        final[entity + '_total_' + str(time)] = final.Recnum.map(temp2.groupby('Recnum')['Amount'].sum())
        final[entity + '_actual/avg_' + str(time)] = final['Amount'] / final[entity + '_avg_' + str(time)]
        final[entity + '_actual/max_' + str(time)] = final['Amount'] / final[entity + '_max_' + str(time)]
        final[entity + '_actual/med_' + str(time)] = final['Amount'] / final[entity + '_med_' + str(time)]
        final[entity + '_actual/total_' + str(time)] = final['Amount'] / final[entity + '_total_' + str(time)]   
        temp2=0
        print(entity + ' amount variables over past ' + str(time) + ' ---> Done')

print('Total run time: {}mins'.format((timeit.default_timer() - start)/60))




Merch description_day_since ---> Done
Merch description_count_0 ---> Done
Merch description amount variables over past 0 ---> Done
Merch description_count_1 ---> Done
Merch description amount variables over past 1 ---> Done
Merch description_count_3 ---> Done
Merch description amount variables over past 3 ---> Done
Merch description_count_7 ---> Done
Merch description amount variables over past 7 ---> Done
Merch description_count_14 ---> Done
Merch description amount variables over past 14 ---> Done
Merch description_count_30 ---> Done
Merch description amount variables over past 30 ---> Done
Run time for the last entity ----------------- 8.591181899999999s

card_merch_day_since ---> Done
card_merch_count_0 ---> Done
card_merch amount variables over past 0 ---> Done
card_merch_count_1 ---> Done
card_merch amount variables over past 1 ---> Done
card_merch_count_3 ---> Done
card_merch amount variables over past 3 ---> Done
card_merch_count_7 ---> Done
card_merch amount variables over p

In [23]:
final.shape

(96397, 515)

In [90]:
final.columns[40:60]

Index(['Merch description_avg_3', 'Merch description_max_3',
       'Merch description_med_3', 'Merch description_total_3',
       'Merch description_actual/avg_3', 'Merch description_actual/max_3',
       'Merch description_actual/med_3', 'Merch description_actual/toal_3',
       'Merch description_count_7', 'Merch description_avg_7',
       'Merch description_max_7', 'Merch description_med_7',
       'Merch description_total_7', 'Merch description_actual/avg_7',
       'Merch description_actual/max_7', 'Merch description_actual/med_7',
       'Merch description_actual/toal_7', 'Merch description_count_14',
       'Merch description_avg_14', 'Merch description_max_14'],
      dtype='object')

In [24]:
new_entities=['Merch description',
       'card_merch', 'card_des', 'card_state', 'card_zip', 'merch_des',
       'merch_state']

In [25]:
start = timeit.default_timer()
# velocity change variables
for ent in new_entities:
    for d in ['0', '1']:
        for dd in ['7', '14', '30']:
            final[ent + '_count_' + d + '_by_' + dd] =\
            final[ent + '_count_' + d]/(final[ent + '_count_' + dd])/float(dd)
            
print('run time: {}s'.format(timeit.default_timer() - start))


run time: 0.11331169999903068s


In [26]:
final.shape

(96397, 557)

In [ ]:
# create benford's law variables

In [27]:
cleaned_data=data[data['Merch description'].str.match(r'[Ff][Ee][Dd][Ee][Xx]')==False].copy()
cleaned_data['Amount']=cleaned_data['Amount'].astype(str)

In [28]:
def n_low(df):
    return ((df['Amount'].str.extract(r'([1-9])')=='1')|(df['Amount'].str.extract(r'([1-9])')=='2')).sum()

In [29]:
cardnums=cleaned_data.groupby('Cardnum').apply(n_low)
cardnums['n']=cleaned_data.groupby('Cardnum')['Amount'].count()
cardnums['n_low']=cleaned_data.groupby('Cardnum').apply(n_low)
cardnums=cardnums[['n','n_low']]
cardnums['n_high']=cardnums['n']-cardnums['n_low']

In [30]:
cardnums['n_low']=cardnums['n_low'].replace(0,1)
cardnums['n_high']=cardnums['n_high'].replace(0,1)

In [31]:
cardnums['R']=1.096*cardnums['n_low']/cardnums['n_high']
cardnums['1/R']=1/cardnums['R']
cardnums['U']=cardnums[['R','1/R']].max(axis=1)

In [32]:
cardnums['t']=(cardnums['n']-15)/3
cardnums['U*']=1+((cardnums['U']-1)/(1+e**-cardnums['t']))

In [33]:
cardnums.sort_values('U*',ascending=False)[0:40].to_csv('Top 40 Cardnum.csv')

In [34]:
merchnums=cleaned_data.groupby('Merchnum').apply(n_low)
merchnums['n']=cleaned_data.groupby('Merchnum')['Amount'].count()
merchnums['n_low']=cleaned_data.groupby('Merchnum').apply(n_low)
merchnums=merchnums[['n','n_low']]
merchnums['n_high']=merchnums['n']-merchnums['n_low']

In [35]:
merchnums['n_low']=merchnums['n_low'].replace(0,1)
merchnums['n_high']=merchnums['n_high'].replace(0,1)

In [36]:
merchnums['R']=1.096*merchnums['n_low']/merchnums['n_high']
merchnums['1/R']=1/merchnums['R']
merchnums['U']=merchnums[['R','1/R']].max(axis=1)

In [37]:
merchnums['t']=(merchnums['n']-15)/3
merchnums['U*']=1+((merchnums['U']-1)/(1+e**-merchnums['t']))

In [38]:
merchnums.sort_values('U*',ascending=False)[0:40].to_csv('Top 40 Merchnum.csv')

In [39]:
cardnums.head()

,n,n_low,n_high,R,1/R,U,t,U*
Cardnum,,,,,,,,
5142110002,1,1,1,1.096,0.912409,1.096000,-4.666667,1.000894
5142110081,4,1,4,0.274,3.649635,3.649635,-3.666667,1.066041
5142110313,3,2,1,2.192,0.456204,2.192000,-4.000000,1.021440
5142110402,11,3,8,0.411,2.433090,2.433090,-1.333333,1.298955
5142110434,1,1,1,1.096,0.912409,1.096000,-4.666667,1.000894


In [40]:
cardnums.index

Int64Index([5142110002, 5142110081, 5142110313, 5142110402, 5142110434,
            5142110651, 5142110691, 5142110749, 5142110909, 5142111097,
            ...
            5142310183, 5142310198, 5142310240, 5142310306, 5142310347,
            5142310397, 5142310525, 5142310598, 5142310768, 5142847398],
           dtype='int64', name='Cardnum', length=1640)

In [41]:
cardnums=cardnums.reset_index()

In [43]:
final_new = pd.merge(final, cardnums[['Cardnum','U*']], how='left',on=['Cardnum'])

In [44]:
merchnums=merchnums.reset_index()

In [45]:
final_new = pd.merge(final_new, merchnums[['Merchnum','U*']], how='left',on=['Merchnum'])

In [46]:
final_new.shape

(96397, 559)

In [47]:
final=final_new

In [48]:
final[['U*_x','U*_y']]=final[['U*_x','U*_y']].fillna(1)

In [49]:
# statistical smoothing
c,nmid=4,20
y_avg=final['Fraud'].mean()
y_dow=final.groupby('dow')['Fraud'].mean()
num=final.groupby('dow').size()
y_dow_smooth=y_avg+(y_dow-y_avg)/(1+np.exp(-(num-nmid)/c))
final['dow_risk']=final['dow'].map(y_dow_smooth)

In [50]:
final.shape

(96397, 560)

In [86]:
# save last 2 months for OOT data


In [51]:
final.to_csv('training_pj3.csv')